In [15]:
class signature_oracle:
    def __init__(self, g, q, N):
        self.g = g
        self.sk = randint(1, N)
    def Sign(self, m):
        return (m, self.g^self.sk)
    def Verify(self, m, sigma):
        return (sigma==self.Sign(m))

In [16]:
# has secret x
class Prover:
    def __init__(self, g, q, N):
        self.honest = True
        self.g = g
        self.q = q
        self.k = randint(1, q)
        self.x = -99999
        
    def Commit(self, S):
        r = self.g^self.k
        return (r, S.Sign(r))
    def Prove(self, S, c):
        s = self.k+c*self.x
        return (s, S.Sign(s))

In [17]:
class Verifier:
    def __init__(self, g, q, N):
        self.g = g
        self.q = q
        
    def Challenge(self):
        c = randint(1,self.q)
        return c
    
    def Verify(self, S, y, r, c, s, sigma_r, sigma_s):
        v1 = S.Verify(r, sigma_r)
        v2 = S.Verify(s, sigma_s)
        v3 = (r==self.g^s * y^(-c))
        return (v1 and v2 and v3)

In [18]:
import time
class SZKP_Protocol:
    def __init__(self, bits, seed):
        start = 0
        end = 0
        
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        
        N = 2^bits
        q = random_prime(N)
        Zq = Integers(q)
        g = Zq(2)
        
        # knowledge
        x = randint(1, q)
        # such that
        y = g^x
        
        prover = Prover(g, q, N)
        verifier = Verifier(g, q, N)
        S = signature_oracle(g, q, N)
        
        if prover.honest:
            prover.x = x
        else:
            prover.x = randint(1, q)

        start = time.time()
        r, sigma_r = prover.Commit(S)
        end = time.time()
        self.commit_time = end-start
        
        start = time.time()
        c = verifier.Challenge()
        end = time.time()
        self.challenge_time = end-start
        
        start = time.time()
        s, sigma_s = prover.Prove(S, c)
        end = time.time()
        self.prove_time = end-start
        
        start = time.time()
        self.result = verifier.Verify(S, y, r, c, s, sigma_r, sigma_s)
        end = time.time()
        self.verify_time = end-start

In [21]:
bits = 2048
ctr = 0
iteration = 20
total_commit_time = 0
total_challenge_time = 0
total_prove_time = 0
total_verify_time = 0

while ctr < iteration:
    seed = time.time_ns()
    print("iteration : %d / %d" % (ctr+1, iteration), end="\r")

    szkp = SZKP_Protocol(bits, seed)
    if szkp.result:
        ctr += 1
        total_commit_time += szkp.commit_time*10^6
        total_challenge_time += szkp.challenge_time*10^6
        total_prove_time += szkp.prove_time*10^6
        total_verify_time += szkp.verify_time*10^6
    else:
        pass

print("\n-----------------------------")
print("Total commit time / Average commit time       : %.2fus / %.2fus" % (total_commit_time, total_commit_time/iteration))
print("Total challenge time / Average challenge time : %.2fus / %.2fus" % (total_challenge_time, total_challenge_time/iteration))
print("Total prove time / Average prove time         : %.2fus / %.2fus" % (total_prove_time, total_prove_time/iteration))
print("Total verify time / Average verify time       : %.2fus / %.2fus" % (total_verify_time, total_verify_time/iteration))


iteration : 20 / 20
-----------------------------
Total commit time / Average commit time       : 158505.68ms / 7925.28ms
Total challenge time / Average challenge time : 396.01ms / 19.80ms
Total prove time / Average prove time         : 77823.64ms / 3891.18ms
Total verify time / Average verify time       : 387604.24ms / 19380.21ms
